In [1]:
from collections import defaultdict
import sys
import os
import pandas as pd
import circRNA as circ

In [2]:
def eprint(*args, **kwargs):
    print(*args,  file=sys.stderr, **kwargs)

def read_circrnas(sample, circrna_bed, nccr_cutoff=10):
    crnas = circ.read_annotation(circrna_bed, fmt="bed")
    selected = (circ for circ in crnas if circ.nccr > nccr_cutoff)
    circ_rnas_exp = [circ.str4diffexp().split("\t") for circ in selected]
    # Je nomme sample la colonne avec le nb_ccr pour préparer la réunion de tous les individus au sine d'un même fichier
    return pd.DataFrame(circ_rnas_exp, columns=['gene',sample]).set_index('gene')

In [3]:
result_dir="../results"
samples_unit = pd.read_csv("../samples12.tsv", sep='\t', dtype=str)
samples = list(set(samples_unit['sample']))

circrnas = []
for sample in samples:
    eprint("reading cirrnas for %s" % sample)
    circrna_bed = os.path.join(result_dir, sample, "circrnas.bed")
    circrnas.append(read_circrnas(sample, circrna_bed, nccr_cutoff=5))

reading cirrnas for cow-testis-adult1
reading cirrnas for cow-testis-neonat2
reading cirrnas for cow-liver-B004
reading cirrnas for cow-liver-B008
reading cirrnas for cow-liver-B005
reading cirrnas for cow-testis-adult3
reading cirrnas for cow-testis-neonat3
reading cirrnas for cow-testis-adult2
reading cirrnas for cow-testis-neonat1


In [4]:
# Ici je réunis tous les dataframe (un par individu) avec comme clef précisément : chr:start-end:strand 
df = circrnas[0].join(circrnas[1:]).fillna(0)

In [5]:
df.to_csv("cirRNAcounts.tsv", sep="\t", index=True)